# Multiset

### _aka_ bag, mset

### _pl._ multizbiór, wielozbiór

**Plan**

1. Definiton of multiset

2. Implementation of multiset

3. Use cases of multiset

# Definition

It's "a modification of the concept of a set that, unlike a set, **allows for multiple instances for each of its elements**". [[source](https://en.wikipedia.org/wiki/Multiset)]

But what _instance of an element_ means? "Instance" is not a programming term here. It means -- more or less -- occurrence, which... also is not very helpful. So let's say that two instances/occurrences an element means that both are **equal to each other**.

In [1]:
# two occurences of `1` are... `1` and `1`, becuase:
assert 1 == 1

# ;D

To put it simply: **multiset is a set that can keep multiple equal elements**. 

So this is a multiset: `⟨1, 1, 2⟩`.

Also, multisets written this way: `⟨1, 1, 2⟩`, `⟨1, 2, 1⟩`, `⟨2, 1, 1⟩`, are identical.

# Implementation

What about the implementation?

In terms of Python's collections, multiset is similar to: 

- `set` (as keeping insertion order is not required)

- `list`/`tuple` (as both keep multiple "equal" elements)

## Multiset as `list` / `tuple`

First option: `list` (or `tuple`). It keeps insertion order, but it is not a requirement for multiset not to keep it.

So `[1, 1, 2, 2, 3]` and `[3, 1, 1, 2, 2]` would be both identical multisets.

**Problems**

1. There is no out-of-the-box way to check for identity (`[1, 2, 1] == [1, 1, 2]` will return `False`).

2. We don't have any (optimized) set operations out-of-the-box.

3. We cannot benfit from set's great feature: constant time (`O(1)`) membership checking.

So generally implementing multiset using `list` or `tuple` sux.

## Multiset as `dict`

Let's try a different approach: a dict with key of multiset element and value of list of all equal elements for a key.

So multiset of `[42, 42, 51, 51, 60]` would be:

In [2]:
 {
     42: [42, 42],
     51: [51, 51],
     60: [60],
 }

{42: [42, 42], 51: [51, 51], 60: [60]}

But why bother building a list of repeated identical elements if we can keep only a count them.

In this implementation multiset of `[41, 41, 52, 52, 60]` would be:

In [3]:
{
    41: 2,
    52: 2,
    60: 1,
}

{41: 2, 52: 2, 60: 1}

We would increment the count on adding new element to multiset and decrement it on removing.

## Multiset as `Counter`

It turns out that we already have this kind of datatype in Python: `Counter`.

In [4]:
from collections import Counter

In [5]:
my_fruit = ['apple', 'banana', 'pear', 'apple', 'apple']

my_fruit_counter = Counter(my_fruit)

my_fruit_counter

Counter({'apple': 3, 'banana': 1, 'pear': 1})

In [6]:
my_fruit_counter['banana'] += 4
my_fruit_counter

Counter({'apple': 3, 'banana': 5, 'pear': 1})

In [7]:
'pear' in my_fruit_counter

True

### Constant time (`O(1)`) membership checking

In [8]:
large_counter = Counter(range(10**6 + 1))
number = 10**6
%timeit number in large_counter

113 ns ± 1.74 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In fact `Counter` inherits from `dict`, so it's not surprising ;)

In [9]:
# compared to list
large_list = list(range(10**6 + 1))
number = 10**6
%timeit number in large_list

11.3 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Counter operations

In [10]:
apple_apple_pear_banana = Counter('apple apple pear banana'.split())
apple_apple_pear_banana

Counter({'apple': 2, 'pear': 1, 'banana': 1})

In [11]:
pear_pear_orange = Counter('pear pear orange'.split())
pear_pear_orange

Counter({'pear': 2, 'orange': 1})

#### Equalily

In [12]:
apple_apple_pear_banana == pear_pear_orange

False

In [13]:
apple_apple_pear_banana == Counter('pear banana apple apple'.split())

True

#### Add

Add counts from two counters.

In [14]:
apple_apple_pear_banana + pear_pear_orange

Counter({'apple': 2, 'pear': 3, 'banana': 1, 'orange': 1})

#### Subtract

Subtract count, but keep only results with positive counts.

In [15]:
apple_apple_pear_banana - pear_pear_orange

Counter({'apple': 2, 'banana': 1})

In [16]:
pear_pear_orange - apple_apple_pear_banana

Counter({'pear': 1, 'orange': 1})

#### Union

Union is the maximum of value in either of the input counters.

In [17]:
apple_apple_pear_banana | pear_pear_orange

Counter({'apple': 2, 'pear': 2, 'banana': 1, 'orange': 1})

#### Intersection

Intersection is the minimum of corresponding counts.

In [18]:
apple_apple_pear_banana & pear_pear_orange

Counter({'pear': 1})

#### Update

Like `dict.update()` but add counts instead of replacing them.


In [19]:
c = Counter('Monty')
c

Counter({'M': 1, 'o': 1, 'n': 1, 't': 1, 'y': 1})

In [20]:
c.update('Python')
c

Counter({'M': 1, 'o': 2, 'n': 2, 't': 2, 'y': 2, 'P': 1, 'h': 1})

#### Enumerating all elements

In [21]:
list((apple_apple_pear_banana.elements()))

['apple', 'apple', 'pear', 'banana']

#### Most common elements

In [22]:
c = Counter('Do you use static type hints in your code?')

In [23]:
c.most_common()

[(' ', 8),
 ('o', 4),
 ('t', 4),
 ('y', 3),
 ('u', 3),
 ('s', 3),
 ('e', 3),
 ('i', 3),
 ('c', 2),
 ('n', 2),
 ('D', 1),
 ('a', 1),
 ('p', 1),
 ('h', 1),
 ('r', 1),
 ('d', 1),
 ('?', 1)]

In [24]:
c.most_common(3)

[(' ', 8), ('o', 4), ('t', 4)]

### Counter pros and cons

#### Pros

- blazingly fast membership checking (vs list/tuple)

- equality checking (vs list/tuple)

- additional operations: `+`, `-`, `&`, `|` (vs dict)

#### Cons

- Counter is a dict, so we cannot store there elements that are not hashable (vs list/tuple).

- It's useless when we want to store all occurrences of equal elements (vs dict).

So what if we want to store all occurrences of equal elements?

In [25]:
class Person:    
    def __init__(self, id_, name, nationality):
        self.id_ = id_
        self.name = name
        self.nationality = nationality
    
    def __hash__(self):
        return hash((self.id_, self.name))
    
    def __eq__(self, other):
        if isinstance(other, Person):
            return self.id_ == other.id_ and self.name == other.name
        return False
    
    def __repr__(self):
        return f'Person({self.id_}, {self.name}, {self.nationality})'

In [26]:
p1 = Person(id_=1, name='Bob', nationality='US')
p2 = Person(id_=1, name='Bob', nationality='UK')
p3 = Person(id_=2, name='Kasia', nationality='PL')
p4 = Person(id_=3, name='Taras', nationality='UA')

In [27]:
Counter([p1, p2, p3, p4])

Counter({Person(1, Bob, US): 2,
         Person(2, Kasia, PL): 1,
         Person(3, Taras, UA): 1})

## Multiset with many occurrences of equal elements

Wrapper on `defaultdict`.

In [28]:
from collections import defaultdict

class Multiset:
    def __init__(self, items=None):
        self.__data = defaultdict(list)
        items = items or []
        for item in items:
            self.__data[hash(item)].append(item)
        
    def __repr__(self):
        return f'Multiset({dict(self.__data)})'
    
    def __contains__(self, other):
        return hash(other) in self.__data
    
    def __eq__(self, other):
        return self.__data == other.__data
        
    def update(self, items):
        for item in items:
            self.__data[hash(item)].append(item)
            
    def remove(self, item):
        self.__data[hash(item)].remove(item)

In [29]:
Multiset('abracadabra')

Multiset({-7035350598420311848: ['a', 'a', 'a', 'a', 'a'], 6180468377144771093: ['b', 'b'], -666094421598522218: ['r', 'r'], -5268657330285807347: ['c'], -7169849235769053121: ['d']})

In [30]:
10 in Multiset([5, 7, 10])

True

In [31]:
m1 = Multiset([10, 21, 21, 32])
m2 = Multiset([10, 21, 21, 32])
m1 == m2

True

In [32]:
print(p1)
print(p2)
print(p3)
print(p4)

Person(1, Bob, US)
Person(1, Bob, UK)
Person(2, Kasia, PL)
Person(3, Taras, UA)


In [33]:
p1 == p2

True

In [34]:
people = Multiset([p1])
people

Multiset({6671638168251787729: [Person(1, Bob, US)]})

In [35]:
people.update([p2])
people

Multiset({6671638168251787729: [Person(1, Bob, US), Person(1, Bob, UK)]})

In [36]:
people.update([p3])
people

Multiset({6671638168251787729: [Person(1, Bob, US), Person(1, Bob, UK)], -8654746187883951607: [Person(2, Kasia, PL)]})

In [37]:
people.remove(p1)
people

Multiset({6671638168251787729: [Person(1, Bob, UK)], -8654746187883951607: [Person(2, Kasia, PL)]})

In [38]:
people.remove(p2)
people

Multiset({6671638168251787729: [], -8654746187883951607: [Person(2, Kasia, PL)]})

### Pros and cons

#### Pros

- Fast membership checking.

- Equality checking.

- We can store multiple occurences of equal items

#### Cos

- No set operations included -- we need to add them ourselves.

- This one is a dict too -- we cannot store there elements that are not hashable (vs list/tuple).

There is a multiset library: https://github.com/wheerd/multiset, but I did not test it (7 stars...).

## Multidict real-life use cases

### Counter

#### Word count

In [77]:
latin_text = """
Respondeo dicendum quod Deum esse quinque viis probari potest. Prima autem et manifestior via est, quae sumitur ex parte motus. Certum est enim, et sensu constat, aliqua moveri in hoc mundo. Omne autem quod movetur, ab alio movetur. Nihil enim movetur, nisi secundum quod est in potentia ad illud ad quod movetur, movet autem aliquid secundum quod est actu. Movere enim nihil aliud est quam educere aliquid de potentia in actum, de potentia autem non potest aliquid reduci in actum, nisi per aliquod ens in actu, sicut calidum in actu, ut ignis, facit lignum, quod est calidum in potentia, esse actu calidum, et per hoc movet et alterat ipsum. Non autem est possibile ut idem sit simul in actu et potentia secundum idem, sed solum secundum diversa, quod enim est calidum in actu, non potest simul esse calidum in potentia, sed est simul frigidum in potentia. Impossibile est ergo quod, secundum idem et eodem modo, aliquid sit movens et motum, vel quod moveat seipsum. Omne ergo quod movetur, oportet ab alio moveri. Si ergo id a quo movetur, moveatur, oportet et ipsum ab alio moveri et illud ab alio. Hic autem non est procedere in infinitum, quia sic non esset aliquod primum movens; et per consequens nec aliquod aliud movens, quia moventia secunda non movent nisi per hoc quod sunt mota a primo movente, sicut baculus non movet nisi per hoc quod est motus a manu. Ergo necesse est devenire ad aliquod primum movens, quod a nullo movetur, et hoc omnes intelligunt Deum. Secunda via est ex ratione causae efficientis. Invenimus enim in istis sensibilibus esse ordinem causarum efficientium, nec tamen invenitur, nec est possibile, quod aliquid sit causa efficiens sui ipsius; quia sic esset prius seipso, quod est impossibile. Non autem est possibile quod in causis efficientibus procedatur in infinitum. Quia in omnibus causis efficientibus ordinatis, primum est causa medii, et medium est causa ultimi, sive media sint plura sive unum tantum, remota autem causa, removetur effectus, ergo, si non fuerit primum in causis efficientibus, non erit ultimum nec medium. Sed si procedatur in infinitum in causis efficientibus, non erit prima causa efficiens, et sic non erit nec effectus ultimus, nec causae efficientes mediae, quod patet esse falsum. Ergo est necesse ponere aliquam causam efficientem primam, quam omnes Deum nominant. Tertia via est sumpta ex possibili et necessario, quae talis est. Invenimus enim in rebus quaedam quae sunt possibilia esse et non esse, cum quaedam inveniantur generari et corrumpi, et per consequens possibilia esse et non esse. Impossibile est autem omnia quae sunt, talia esse, quia quod possibile est non esse, quandoque non est. Si igitur omnia sunt possibilia non esse, aliquando nihil fuit in rebus. Sed si hoc est verum, etiam nunc nihil esset, quia quod non est, non incipit esse nisi per aliquid quod est; si igitur nihil fuit ens, impossibile fuit quod aliquid inciperet esse, et sic modo nihil esset, quod patet esse falsum. Non ergo omnia entia sunt possibilia, sed oportet aliquid esse necessarium in rebus. Omne autem necessarium vel habet causam suae necessitatis aliunde, vel non habet. Non est autem possibile quod procedatur in infinitum in necessariis quae habent causam suae necessitatis, sicut nec in causis efficientibus, ut probatum est. Ergo necesse est ponere aliquid quod sit per se necessarium, non habens causam necessitatis aliunde, sed quod est causa necessitatis aliis, quod omnes dicunt Deum. Quarta via sumitur ex gradibus qui in rebus inveniuntur. Invenitur enim in rebus aliquid magis et minus bonum, et verum, et nobile, et sic de aliis huiusmodi. Sed magis et minus dicuntur de diversis secundum quod appropinquant diversimode ad aliquid quod maxime est, sicut magis calidum est, quod magis appropinquat maxime calido. Est igitur aliquid quod est verissimum, et optimum, et nobilissimum, et per consequens maxime ens, nam quae sunt maxime vera, sunt maxime entia, ut dicitur II Metaphys. Quod autem dicitur maxime tale in aliquo genere, est causa omnium quae sunt illius generis, sicut ignis, qui est maxime calidus, est causa omnium calidorum, ut in eodem libro dicitur. Ergo est aliquid quod omnibus entibus est causa esse, et bonitatis, et cuiuslibet perfectionis, et hoc dicimus Deum. Quinta via sumitur ex gubernatione rerum. Videmus enim quod aliqua quae cognitione carent, scilicet corpora naturalia, operantur propter finem, quod apparet ex hoc quod semper aut frequentius eodem modo operantur, ut consequantur id quod est optimum; unde patet quod non a casu, sed ex intentione perveniunt ad finem. Ea autem quae non habent cognitionem, non tendunt in finem nisi directa ab aliquo cognoscente et intelligente, sicut sagitta a sagittante. Ergo est aliquid intelligens, a quo omnes res naturales ordinantur ad finem, et hoc dicimus Deum.
""".split()

In [78]:
c = Counter(latin_text)
c.most_common(15)

[('quod', 35),
 ('est', 34),
 ('et', 32),
 ('in', 30),
 ('non', 22),
 ('aliquid', 14),
 ('autem', 13),
 ('esse', 10),
 ('quae', 9),
 ('hoc', 9),
 ('per', 9),
 ('causa', 8),
 ('ex', 7),
 ('enim', 7),
 ('a', 7)]

#### Shopping cart

In [40]:
Counter('butter chocolate chocolate milk milk chocolate butter'.split())

Counter({'butter': 2, 'chocolate': 3, 'milk': 2})

In [79]:
# merging carts can be useful

Counter('butter milk milk chocolate butter'.split()) + Counter('chocolate chocolate chocolate'.split())

Counter({'butter': 2, 'milk': 2, 'chocolate': 4})

http://www.java2s.com/Tutorial/Cpp/0380__set-multiset/Catalog0380__set-multiset.htm ???

### Multidict with copies

#### When we want to store multiple hashable elements with a mutable part. 

In [48]:
...

Ellipsis